In [0]:
from google.colab import files
uploaded = files.upload()

Saving testData.csv to testData.csv
Saving testLabels.csv to testLabels.csv
Saving trainData1.csv to trainData1.csv
Saving trainData2.csv to trainData2.csv
Saving trainData3.csv to trainData3.csv
Saving trainData4.csv to trainData4.csv
Saving trainData5.csv to trainData5.csv
Saving trainData6.csv to trainData6.csv
Saving trainData7.csv to trainData7.csv
Saving trainData8.csv to trainData8.csv
Saving trainData9.csv to trainData9.csv
Saving trainData10.csv to trainData10.csv
Saving trainLabels1.csv to trainLabels1.csv
Saving trainLabels2.csv to trainLabels2.csv
Saving trainLabels3.csv to trainLabels3.csv
Saving trainLabels4.csv to trainLabels4.csv
Saving trainLabels5.csv to trainLabels5.csv
Saving trainLabels6.csv to trainLabels6.csv
Saving trainLabels7.csv to trainLabels7.csv
Saving trainLabels8.csv to trainLabels8.csv
Saving trainLabels9.csv to trainLabels9.csv
Saving trainLabels10.csv to trainLabels10.csv


In [0]:
#!/usr/bin/python3
import csv
import glob
import numpy as np
import math

import sys
np.set_printoptions(threshold=sys.maxsize)

trainDataPath = glob.glob("./trainData*.csv") # Get path for TrainData
trainLabelPath = glob.glob("./trainLabels*.csv")	# Get path for TrainLabels

testDataPath = "./testData.csv"
testLabelPath = "./testLabels.csv"

#x_values = np.array(range(1, 31))
#y_values=[]

def readInData(file, arrayToPopulate):
    ''' Parameters:
        dataPath: CSV file to extract from
           arrayToPopulate: Array to add values to'''
    with open(file, "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            arrayToPopulate.append(row)

def convertToIntNumpy(dataToChange):
    for i in range(len(dataToChange)):
        vector = list(map(float, dataToChange[i]))
        dataToChange[i] = np.array(vector)

    dataToChange = np.array(dataToChange)

def mix():
    testData = []
    testLabel = []
    trainData = []
    trainLabel = []

    for dataFile in trainDataPath: # Read in all the training data
        readInData(dataFile, trainData)
    for labelFile in trainLabelPath: # Read in all the training labels
        readInData(labelFile, trainLabel)

    readInData(testDataPath, testData) # Read in the test data
    readInData(testLabelPath, testLabel) # Read in the test label

    convertToIntNumpy(trainData)
    convertToIntNumpy(trainLabel)
    convertToIntNumpy(testData)
    convertToIntNumpy(testLabel)

    # Label 5 is class 1, y_n = 1
    # Label 6 is class 2, y_n = 0
    # Calculate pi
    N = len(trainLabel)
    y_n = np.array([])
    for row in trainLabel:
        if (row[0] == 5):
            y_n = np.append(y_n, 1)
        elif (row[0] == 6):
            y_n = np.append(y_n, 0)
    N1 = y_n.sum() # number of data points in label 5 (class 1)
    N2 = N - N1 # number of data points in label 6 (class 0)
    pi = N1 / N

    u_1 = np.matrix([0] * 64, float)
    u_2 = np.matrix([0] * 64, float)
    for i in range(len(trainData)):
        if (y_n[i] == 1):
            u_1 = np.add(u_1, trainData[i])
        elif (y_n[i] == 0):
            u_2 = np.add(u_2, trainData[i])

    u_1 = u_1 / N1
    u_2 = u_2 / N2

    S1 = np.matrix([0] * 64, float) # Make 64 x 64 matrix
    S2 = np.matrix([0] * 64, float)
    print(trainData[0])
    print(u_1)
    print((trainData[0] - u_1).transpose())
    print((trainData[0] - u_1).shape())
    print(np.matmul((trainData[0] - u_1).transpose(),trainData[0] - u_1))

    for vector in trainData:
        S1 = np.add(S1, np.matmul((vector - u_1).transpose(),(vector - u_1)))
        S2 = np.add(S2, np.matmul((vector - u_2).transpose(),(vector - u_2)))
    sigma = np.add((N1/N) * S1, (N2/N) * S2)
    sigmaInverse = np.linalg.inv(sigma)


    w = np.matmul(sigmaInverse, (u_1 - u_2).transpose()) #Transpose not in the same order since I am reading in in rows
    w_0 = np.add(np.add(-0.5 * np.matmul(np.matmul(u_1, sigmaInverse), u_1.transpose()),
        0.5 * np.matmul(np.matmul(u_2, sigmaInverse), u_2.transpose())),
        math.log(pi / (1 - pi)))
    accuracy = 0
    for i in range(len(testData)):
        exponentialInput = -(np.matmul(testData[i], w).item(0) + w_0.item(0))
        sigmoidInput = math.exp(exponentialInput)
        sigmoid = 1 / (1 + sigmoidInput)
        predicted = ""
        label = testLabel[i][0]
        if (sigmoid >= 0.5):
            predicted = 5.0
        elif (sigmoid < 0.5):
            predicted = 6.0
        if (label == predicted): accuracy += 1
mix()

[0. 0. 0. 5. 9. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 6. 3. 0. 0. 0. 0. 2. 0. 7. 3. 6. 0. 0. 0. 0. 0. 0. 0. 7. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[ 4.802  4.99   6.758  9.676  9.394  7.414  5.574  4.6    4.692  5.418
   9.086 10.038  7.73   6.356  4.786  4.52   4.734  5.468  9.762  7.648
   5.292  4.756  4.552  4.548  4.688  5.882 10.436  8.616  6.862  5.916
   4.87   4.682  4.648  5.512  8.98   8.712  9.038  8.39   5.51   4.376
   4.7    5.242  7.734  7.774  7.608  8.944  7.03   4.512  4.434  4.886
   7.958  9.016  7.96   8.706  7.138  4.708  4.698  4.7    7.064  9.652
   9.644  8.122  5.736  4.774]]
[[ -4.802]
 [ -4.99 ]
 [ -6.758]
 [ -4.676]
 [ -0.394]
 [ -7.414]
 [ -5.574]
 [ -4.6  ]
 [ -4.692]
 [ -5.418]
 [ -9.086]
 [-10.038]
 [ -7.73 ]
 [ -6.356]
 [ -4.786]
 [ -4.52 ]
 [ -4.734]
 [ -5.468]
 [ -9.762]
 [ -7.648]
 [ -5.292]
 [ -4.756]
 [ -4.552]
 [ -4.548]
 [ -4.688]
 [  0.118]
 [ -7.436]
 [ -8.616]
 [ -6.862]
 [ -5.916]
 [ -4.87 ]

TypeError: ignored